# Computing Per-Transaction Responsibility for Ethereum

Ethereum miners use energy that causes emissions. The amount of emissions varies with respect to energy sources, and the about of energy varies mainly as a function of time with respect to GPU efficiency. To simplify this discussion, we will work with hashrate (hashes per second) and hashcount (hashes) instead of energy or emissions.

We assume:

* It is more meaningful to allocate hashcount responsibility across all miners, or all transactions, than across both simultaneously. Here we allocate based on transactions, because the goal is to create accountability at the point of use.
* The hashrate does not vary directly based on the properties of the transactions. This means allocating a hashcount to each transaction is a philosophical question rather than a technical question.
* The sum of allocated usage should add up to the total usage. This means the sum of transaction hashcounts should add up to the total Ethereum hashcount. Or that the sum of all allocated emissions should add up to the total emissions.
* Transactions contribute to the hashrate in proportion to how much the transaction incentivizes miners.

Miners are incentivized by payments in Ethereum, and by speculation about the future value of Ethereum. Miners are paid in two ways:

1. Varying transaction fees paid by users when making a transaction, recently around 2ETH total.
2. A fixed per-block reward at 2ETH (which was previously 3ETH and 5ETH), with additional nuances for uncle blocks.

There are a few big questions to keep in mind:

- Should responsibility be allocated based on first order effects (like payment of transaction fees) or second order effects (like increasing the hashrate)? If the hashrate of Ethereum was fixed, how could we allocate responsibility?
- Should responsibility be assigned as a portion of a block's energy usage (assuming each block is the same throughout a day), a portion of one day, or as a portion of a longer timeline like the entire history of Ethereum? Should a transaction years ago also be considered partially responsible for energy usage today?
- Should the block rewards be ignored, or should they factor in to the model evenly across all transactions, based on gas usage, or something else?
- When comparing over longer timelines, how can we account for the fact that a higher price for Ethereum provides a higher incentive to miners?

The following models explore the effects of different answers to these questions.

## Preparation

First we download some daily statistics about Ethereum, and load them into dictionaries.

In [1]:
!wget --quiet -O "data/statistics.csv" "https://docs.google.com/spreadsheets/d/e/2PACX-1vRigeH5xVkOZNOtBM6blHfrPXNqyWrtS7fVzOUm_d9fbNtTyTCBwAnbBPdQ76q6-2p0xoNcy7VxhXnZ/pub?output=csv"

In [2]:
import pandas as pd

class EthereumHistory:
    def __init__(self, fn='data/statistics.csv'):
        df = pd.read_csv(fn)
        
        dates = [e.date() for e in pd.to_datetime(df['Date'])]
        self.dates = dates
        
        self.tx_count = dict(zip(dates, df['Tx count']))
        self.tx_fees = dict(zip(dates, df['Tx fees (ETH)']))
        self.block_count = dict(zip(dates, df['Block count']))
        self.block_rewards = dict(zip(dates, df['Block rewards (ETH)']))
        self.gas_used = dict(zip(dates, df['Gas used']))
        self.price = dict(zip(dates, df['Price (USD)']))
        self.hashcount = dict(zip(dates, df['Hashcount (EH/day)']))
        
        self.tx_count_total = sum(self.tx_count.values())
        self.tx_fees_total = sum(self.tx_fees.values())
        self.block_count_total = sum(self.block_count.values())
        self.block_rewards_total = sum(self.block_rewards.values())
        self.gas_used_total = sum(self.gas_used.values())
        self.price_total = sum(self.price.values())
        self.hashcount_total = sum(self.hashcount.values())
        
history = EthereumHistory()

Some helper functions for simplifying the code behind different models.

In [3]:
from etherscan import Etherscan, etherscan_gas_fees, etherscan_gas_used, etherscan_timestamp
from nifty_gateway import list_nifty_gateway
from utils import load_contracts

def get_tx_date(tx):
    return etherscan_timestamp(tx).date()
    
def get_tx_fees(tx):
    return etherscan_gas_fees(tx) / 1e18 # convert wei to eth

def get_tx_gas(tx):
    return etherscan_gas_used(tx)

## Simple models

Initial discussion around allocating energy and emissions was focused on [per-transaction allocation](https://digiconomist.net/ethereum-energy-consumption):

$
hashcount_{date} \times \dfrac{1}{txcount_{date}}
$

Or [gas-weighted allocation](https://memoakten.medium.com/the-unreasonable-ecological-cost-of-cryptoart-2221d3eb2053):

$
hashcount_{date} \times \dfrac{gas_{tx}}{gas_{date}}
$

We repeat that here.

In [4]:
def model_tx_count(tx):
    date = get_tx_date(tx)
    return history.hashcount[date] / history.tx_count[date]

def model_gas(tx):
    date = get_tx_date(tx)
    return history.hashcount[date] * get_tx_gas(tx) / history.gas_used[date]

## FeesOnly model

This is the simplest interpretation of the idea that "users should be held accountable for a service proportional to how much they pay for that service", because the payment is what rewards and therefore incentivizes the service to continue running. This is similar to the gas-based model, adding gas price to the mix.

$
hashcount_{date}\times\dfrac{fees_{tx}}{fees_{date}}
$

In [5]:
def model_fees_only(tx):
    tx_date = get_tx_date(tx)
    fee_ratio = get_tx_fees(tx) / history.tx_fees[tx_date]
    return history.hashcount[tx_date] * fee_ratio

## HistoricalFees model

The approach of the FeesOnly model looks at each transaction in the context of the day on which it was made. This time envelope could be narrowed or expanded. It could be narrowed to the block on which it was made (which should be similar to FeesOnly with some added noise). Or it could be expanded to include the entire history of Ethereum. Expanding would account for the fact that Ethereum transactions do not happen in a vacuum, but they build on all previous transactions, and lay a foundation for future transactions.

One challenge of this model: until Ethereum docks with Ethereum 2, these numbers will changes over time as the `history.hashcount_total` and `history.fees_total` changes.

$
hashcount_{total}\times\dfrac{fees_{tx}}{fees_{total}}
$

In [6]:
def model_historical_fees(tx):
    fee_ratio = get_tx_fees(tx) / history.tx_fees_total
    return history.hashcount_total * fee_ratio

## HistoricalFeesTimesPrice model

Because the price of Ethereum varies from day to day, a transaction fee on a high-priced day pays more to miners in fiat equivalent than the same transaction fee on a low-priced day.

$
hashcount_{total}\times\dfrac{fees_{tx} \times price_{date}}{\sum_{d}^{history} fees_{d} \times price_{d}}
$

In [7]:
# compute the fee volume across all dates 
all_fees_price = sum([history.tx_fees[date] * history.price[date] for date in history.dates])

def model_historical_fees_price(tx):
    tx_date = get_tx_date(tx)
    tx_fees_price = get_tx_fees(tx) * history.price[tx_date]
    fees_price_ratio = tx_fees_price / all_fees_price
    return history.hashcount_total * fees_price_ratio

## FeesAndBlock model

It could be argued that miners are only half-incentivized by transaction fees, because transaction fees only make up about half of their income, with the other half coming from block rewards.

Consider two blocks with two transactions each:

1. Block 1 has transaction A paying 200 gwei and transaction B paying 100 gwei
2. Block 2 has transaction A paying 2 ETH and transaction B paying 1 ETH

Although the ratio between transactions A and B are similar in each block, the miner is arguably disproportionately more incentivized by transaction A over transaction B in block 2 compared to block 1. If the block reward is 2 ETH, and we assume that every transaction is equally responsible for the block reward, then we could argue that the weighting for the transactions in each block is as follows:

* block 1 tx A = (200 gwei + 2 ETH / 2) / (200 gwei + 100 gwei + 2 ETH) ~= 1/2
* block 1 tx B = (100 gwei + 2 ETH / 2) / (200 gwei + 100 gwei + 2 ETH) ~= 1/2
* block 2 tx A = (2 ETH + 2 ETH / 2) / (2 ETH + 1 ETH + 2 ETH) = 4/6
* block 2 tx B = (1 ETH + 2 ETH / 2) / (2 ETH + 1 ETH + 2 ETH) = 2/6

Like other models, this model operates on a per-day basis, so the block rewards include uncle block rewards.

$
hashcount_{date}\times\dfrac{fees_{tx} + blockrewards_{date} \times \dfrac{1}{txcount_{date}}}{fees_{date} + blockrewards_{date}}
$

In [8]:
def model_fees_block(tx):
    tx_date = get_tx_date(tx)
    block_rewards = history.block_rewards[tx_date]
    block_rewards_portion = block_rewards / history.tx_count[tx_date]
    tx_fees_and_block = get_tx_fees(tx) + block_rewards_portion
    all_fees_and_block = history.tx_fees[tx_date] + block_rewards
    fees_and_block_ratio = tx_fees_and_block / all_fees_and_block
    return history.hashcount[tx_date] * fees_and_block_ratio

## FeesAndBlockTimesGas model

Instead of assuming every transaction is equally responsible for the block reward, we can use gas as a weighting factor. This makes sense because high-gas transactions effectively "push out" low-gas transactions. If one transaction used 12,000,000 gas, then it uses part of the block that could be otherwised used for 120x transactions that use 100,000 gas.

$
hashcount_{date}\times\dfrac{fees_{tx} + blockrewards_{date} \times \dfrac{gas_{tx}}{gas_{date}}}{fees_{date} + blockrewards_{date}}
$

In [9]:
def model_fees_block_gas(tx):
    tx_date = get_tx_date(tx)
    block_rewards = history.block_rewards[tx_date]
    tx_gas_ratio = get_tx_gas(tx) / history.gas_used[tx_date]
    block_rewards_portion = block_rewards * tx_gas_ratio
    tx_fees_and_block = get_tx_fees(tx) + block_rewards_portion
    all_fees_and_block = history.tx_fees[tx_date] + block_rewards
    fees_and_block_ratio = tx_fees_and_block / all_fees_and_block
    return history.hashcount[tx_date] * fees_and_block_ratio

## HistoricalFeesAndBlock model

This opens the "time envelope" of the FeesAndBlock approach to cover all of Ethereum's history.

$hashcount_{total}\times\dfrac{fees_{tx} + blockrewards_{total} \times \dfrac{1}{txcount_{total}}}{fees_{total} + blockrewards_{total}}$

In [10]:
def model_historical_fees_block(tx):
    block_rewards = history.block_rewards_total
    block_rewards_portion = block_rewards / history.tx_count_total
    tx_fees_and_block = get_tx_fees(tx) + block_rewards_portion
    all_fees_and_block = history.tx_fees_total + block_rewards
    fees_and_block_ratio = tx_fees_and_block / all_fees_and_block
    return history.hashcount_total * fees_and_block_ratio

## HistoricalFeesAndBlockTimesGas model

This adds gas weighting to the HistoricalFeesAndBlock model.

$hashcount_{total}\times\dfrac{fees_{tx} + blockrewards_{total} \times \dfrac{gas_{tx}}{gas_{total}}}{fees_{total} + blockrewards_{total}}$

In [11]:
def model_historical_fees_block_gas(tx):
    block_rewards = history.block_rewards_total
    tx_gas_ratio = get_tx_gas(tx) / history.gas_used_total
    block_rewards_portion = block_rewards * tx_gas_ratio
    tx_fees_and_block = get_tx_fees(tx) + block_rewards_portion
    all_fees_and_block = history.tx_fees_total + block_rewards
    fees_and_block_ratio = tx_fees_and_block / all_fees_and_block
    return history.hashcount_total * fees_and_block_ratio

## HistoricalFeesAndBlockTimesPrice model

This weights HistoricalFeesAndBlock by the price of Ethereum.

$hashcount_{total}\times\dfrac{(fees_{tx} + blockrewards_{total} \times \dfrac{1}{txcount_{total}}) \times price_{date}}{\sum_{d}^{history} (fees_{d} + blockrewards_{d}) \times price_{d}}$

In [12]:
all_fees_block_price = \
    sum([(history.tx_fees[date] + history.block_rewards[date]) * history.price[date] 
         for date in history.dates])

def model_historical_fees_block_price(tx):
    tx_price = history.price[get_tx_date(tx)]
    block_rewards = history.block_rewards_total
    block_rewards_portion = block_rewards / history.tx_count_total
    tx_fees_and_block_price = (get_tx_fees(tx) + block_rewards_portion) * tx_price
    fees_and_block_price_ratio = tx_fees_and_block_price / all_fees_block_price
    return history.hashcount_total * fees_and_block_price_ratio

## HistoricalFeesAndBlockTimesGasAndPrice model

This weights HistoricalFeesAndBlockTimesPrice by the gas used instead of treating each transaction equally.

$hashcount_{total}\times\dfrac{(fees_{tx} + blockrewards_{total} \times \dfrac{gas_{tx}}{gas_{total}}) \times price_{date}}{\sum_{d}^{history} (fees_{d} + blockrewards_{d}) \times price_{d}}$

In [13]:
def model_historical_fees_block_gas_price(tx):
    tx_price = history.price[get_tx_date(tx)]
    block_rewards = history.block_rewards_total
    tx_gas_ratio = get_tx_gas(tx) / history.gas_used_total
    block_rewards_portion = block_rewards * tx_gas_ratio
    tx_fees_and_block_price = (get_tx_fees(tx) + block_rewards_portion) * tx_price
    fees_and_block_price_ratio = tx_fees_and_block_price / all_fees_block_price
    return history.hashcount_total * fees_and_block_price_ratio

The last two models are harder to implement because they require close analysis of the Ethereum blockchain.

## ExtraPrice model

Miners prefer transactions that pay higher gas prices. To the extent that a transaction is higher than the minimum gas price that might have been paid, we may consider that transaction to be disproportionately incentivizing miners, and therefore disproportionately responsible for energy use. Something like:

```python
def model_extra_fee(tx):
    block_tx = get_transactions(get_block(tx))
    gas_prices = [get_gas_price(e) for e in block_tx]
    min_gas_price = min(gas_prices)
    tx_weight = get_gas_price(tx) - min_gas_price
    all_weights = sum([(get_gas_price(e) - min_gas_price) for e in block_tx])
    block_hashcount = history.hashcount[date] / history.block_count[date]
    return block_hashcount * tx_weight / all_weights
```

## ExtraFee model

Instead of looking at the increase over minimum gas price alone, we can also consider that a large gas usage with a larger gas price pays more to miners than a low gas usage with the same gas price. This could be considered the "extra" beyond what the miner would have been "guaranteed" to get paid otherwise. Something like:

```python
def model_extra_fee(tx):
    block_tx = get_transactions(get_block(tx))
    gas_prices = [get_gas_price(e) for e in block_tx]
    min_gas_price = min(gas_prices)
    tx_extra = get_tx_fees(tx) - min_gas_price * get_tx_gas(tx)
    all_extra = sum([(get_tx_fees(e) - min_gas_price * get_tx_gas(e)) for e in block_tx])
    block_hashcount = history.hashcount[date] / history.block_count[date]
    return block_hashcount * tx_extra / all_extra
```

Both of these techniques have two related problems:

1. Any transaction that has the lowest gas price will be held unaccountable.
2. If all transactions have the same gas price, the result is undefined.

## Test on one contract

Let's test all the methods on a single contract. First we load the transactions then compute the different model results. Results are in exahashes (EH). For context, an average day on Ethereum at the beginning of 2021-04 was 42EH/day.

In [14]:
methods = [
    model_tx_count,
    model_gas,
    model_fees_only,
    model_historical_fees,
    model_historical_fees_price,
    model_fees_block,
    model_fees_block_gas,
    model_historical_fees_block,
    model_historical_fees_block_gas,
    model_historical_fees_block_price,
    model_historical_fees_block_gas_price
]

contracts = load_contracts()
etherscan = Etherscan()
address = contracts['Foundation/ERC-721']
transactions = etherscan.load_transactions(address)
for method in methods:
    energy = sum(map(method, transactions))
    print(f'{energy:4.1f}EH {method.__name__}')

 2.2EH model_tx_count
 4.3EH model_gas
 3.7EH model_fees_only
 8.5EH model_historical_fees
18.0EH model_historical_fees_price
 2.9EH model_fees_block
 4.1EH model_fees_block_gas
 2.1EH model_historical_fees_block
 3.6EH model_historical_fees_block_gas
13.9EH model_historical_fees_block_price
23.8EH model_historical_fees_block_gas_price


## Run on all contracts

Finally we run all methods on all contracts.

In [15]:
cols = ('Name', 'Kind', *[e.__name__ for e in methods])

rows = []
for name_kind, address in contracts.items():
    if name_kind == 'Nifty Gateway/multiple':
        addresses = list_nifty_gateway(update=False)
        transactions = etherscan.load_transactions_multiple(addresses)
    else:
        transactions = etherscan.load_transactions(address)
        
    row = name_kind.split('/')
    for method in methods:
        energy = sum(map(method, transactions))
        row.append(energy)
    rows.append(row)
    
df = pd.DataFrame(rows, columns=cols)
df

,Name,Kind,model_tx_count,model_gas,model_fees_only,model_historical_fees,model_historical_fees_price,model_fees_block,model_fees_block_gas,model_historical_fees_block,model_historical_fees_block_gas,model_historical_fees_block_price,model_historical_fees_block_gas_price
0,Async,ASYNC,0.143748,0.162558,0.124107,0.139823,0.067317,0.143883,0.159391,0.172663,0.219798,0.163095,0.208760
1,Async,ASYNC-V2,0.200851,0.344032,0.296351,0.709047,0.940308,0.239131,0.323761,0.237860,0.396614,0.838347,1.370288
2,Foundation,ERC-721,2.164738,4.345547,3.725619,8.474106,18.038222,2.883011,4.056971,2.119077,3.630994,13.865983,23.775195
3,Foundation,FND NFT (FNDNFT) ERC-20,0.892060,3.661544,3.048163,6.859674,14.622932,1.881579,3.376217,1.109844,3.030809,7.278497,19.887193
4,Known Origin,KnownOriginDigitalAsset (KODA),0.494542,1.785069,1.121553,1.695370,2.455538,0.680474,1.713380,0.574115,1.885666,1.479002,4.087416
5,Makersplace,MakersTokenV2 (MKT2),1.737923,8.157515,5.467773,9.116728,14.254929,2.980914,7.812724,2.293773,8.939848,6.699187,22.346666
6,Nifty Gateway,multiple,4.166663,17.908292,19.258833,49.140657,95.725269,11.150536,18.546008,6.780348,17.766044,38.859482,99.169371
7,Nifty Gateway,GUSD cashout,0.665149,0.519574,0.587835,1.446186,2.869845,0.630255,0.549573,0.615913,0.498730,3.559316,2.942581
8,OpenSea,Wyvern Exchange,20.732939,59.701787,43.841142,77.360625,138.746225,28.929100,57.856826,23.098735,59.997372,88.764434,190.863013
9,OpenSea,OpenSeaENSResolver,0.050202,0.032034,0.017652,0.004431,0.001010,0.049057,0.031526,0.047421,0.033440,0.031388,0.022623


## Disclaimers

* Analyzing hashcount (and implicitly, energy usage and emissions) is only half of a cost-benefit analysis. To determine whether Ethereum as a whole is an efficient use of energy, it is more appropriate to compare the costs and benefits of Ethereum to the costs and benefits of other markets or industries. This work only evaluates the costs.
* To compute energy or emissions instead of hashcount, `hashcount` must be substituted with `energy` or `emissions` in all models. Neither energy nor emissions can be computed from hashcount directly because hashing efficiency (hashes per Watt) and emissions intensity (CO2 per Watt) vary over time.